In [3]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
import numpy as np
from bert.tokenization import FullTokenizer
from tqdm import tqdm_notebook
from tensorflow.keras import backend as K

ModuleNotFoundError: No module named 'tqdm'

In [6]:
df= pd.read_csv('data_5_products.csv', index_col='Complaint ID', usecols=['Product','Sub-product', 'Issue', 'Sub-issue', 'Consumer complaint narrative', 'Complaint ID'])

In [7]:
df.head()

,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative
Complaint ID,,,,,
1297939,Credit card,NaN,Other,NaN,Received Capital One charge card offer XXXX. A...
1296693,Credit card,NaN,Rewards,NaN,I 'm a longtime member of Charter One Bank/RBS...
1296727,Debt collection,Payday loan,Improper contact or sharing of info,Talked to a third party about my debt,I received a call from a XXXX XXXX from XXXX @...
1296785,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,I had my purse stolen in 2007. They never foun...
1295146,Debt collection,Mortgage,Communication tactics,Threatened to take legal action,"Going through a divorce, my ex and I were unab..."


In [8]:
df.count()

Product                         36958
Sub-product                     20302
Issue                           36958
Sub-issue                       15661
Consumer complaint narrative    36958
dtype: int64

In [10]:
df['Sub-product'].value_counts()

Checking account                          4276
Conventional fixed mortgage               3226
Other (i.e. phone, health club, etc.)     2164
Other bank product/service                1378
FHA mortgage                              1343
Credit card                               1338
Conventional adjustable mortgage (ARM)    1203
I do not know                             1174
Medical                                   1120
Other mortgage                             693
Home equity loan or line of credit         465
Payday loan                                401
Savings account                            344
VA mortgage                                343
Mortgage                                   208
Auto                                       169
Non-federal student loan                   129
(CD) Certificate of deposit                125
Federal student loan                        94
Reverse mortgage                            62
Cashing a check without an account          47
Name: Sub-pro

In [24]:
x=df[df['Product']=='Credit card']
x.count()

Product                         36958
Sub-product                     20302
Issue                           36958
Sub-issue                       15661
Consumer complaint narrative    36958
dtype: int64

In [22]:
df.count()

Product                         36958
Sub-product                     20302
Issue                           36958
Sub-issue                       15661
Consumer complaint narrative    36958
dtype: int64